In [ ]:
# To interface with Excel
library(openxlsx)

# For data manipulation & visualization
library(tidyverse)

# For "tidying" regression output
library(broom)

# Borrow a tidyverse dataset
head(diamonds)

### The `data` worksheet

First we'll set up objects referring to our workbook and worksheet, respectively. 

In [ ]:
# Add workbook
wb <- createWorkbook()

# Add worksheet of raw data
data_sheet <- addWorksheet(wb, sheetName = 'data')

Next, we'll use the `writeDataTable()` function to write `diamonds` to this worksheet as an Excel table. 

In [ ]:
# Exclude row names
writeDataTable(wb, sheet = data_sheet, x = diamonds, rowNames = FALSE)

### The `analysis` worksheet

I would also like to include an `analysis` worksheet including regression results and a scatterplot of the relationship between the log of `price` and the log of `carat`. 

We can set up a new worksheet, add the results of the regression with `writeData()` and add the scatterplot with `writePlot()` 

In [ ]:
# Set up a new worksheet 
analysis_sheet <- addWorksheet(wb, sheetName = 'analysis')

In [ ]:
# Untransformed relationship
ggplot(data = diamonds, aes(x = carat, y = price)) +
    geom_point() 

In [ ]:
# Log transformations
diamonds <- diamonds %>% 
  mutate(carat_log = log(carat)) %>% 
  mutate(price_log = log(price))

In [ ]:
# Transformed relationship
ggplot(data = diamonds, aes(x = carat_log, y = price_log)) +
    geom_point() 

In [ ]:
# Linear regression -- price "by" carat
diamonds_lm <- lm(price ~ carat, data = diamonds)

# Clean up the regression output
diamonds_lm_tidy <- tidy(diamonds_lm)
diamonds_lm_tidy


In [ ]:
# Add the table to this worksheet
writeDataTable(wb, sheet = analysis_sheet, x = diamonds_lm_tidy, rowNames = FALSE)

In [ ]:
# Visualize the results with regression trendline
ggplot(data = diamonds, aes(x = carat_log, y = price_log)) +
    geom_point() +
    geom_smooth(method = 'lm')


# Add the image to the worksheet, set the placement
insertPlot(wb, sheet = analysis_sheet, startRow = 5, startCol = "A", width = 4, height = 4)

### Admire our work, part 1

OK, let's save and view our work. By default, `saveWorkbook()` will put the file in our working directory.

Let's take a look at the finished output in Excel.

In [ ]:
diamonds_report = 'output/diamonds-report.xlsx'

# save workbook to working directory
saveWorkbook(wb = wb, file = diamonds_report, overwrite = TRUE)

It's pretty good! But as a matter of fact I would like to do a few more things to the workbook:

- Freeze panes
- Format the `price` as currency in the `data` worksheet. 

Let's give it a go:

In [ ]:
# Freeze panes

for (sheet in getSheetNames(diamonds_report)){
    freezePane(wb, sheet = sheet, firstRow = TRUE)
}

In [ ]:
# Set currency style to price column

currency <- createStyle(numFmt = "$#,##0")
addStyle(wb, sheet = data_sheet, style = currency, cols = which(colnames(diamonds) == 'price'), 2:(nrow(mpg) + 1))

Let's try this again!

In [ ]:
saveWorkbook(wb = wb, file = diamonds_report, overwrite = TRUE)

# See you back at the slides for the conclusion